In [7]:
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
import numpy as np
from rasterio.plot import show
from rasterio.mask import mask
from matplotlib.colors import ListedColormap
from matplotlib import rcParams
from matplotlib.patches import Rectangle

from matplotlib.mathtext import _mathtext as mathtext

mathtext.FontConstantsBase.sup1 = 0.40
rcParams['font.family'] = 'serif'
rcParams['font.serif'] = ['Arial']
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.rm'] = 'Arial'
plt.rcParams['mathtext.it'] = 'Arial'
plt.rcParams['mathtext.bf'] = 'Arial'


In [8]:
import rasterio
from rasterio.plot import show
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.patches import Rectangle

def hex_to_rgb(hex_color):

    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16)/255 for i in (0, 2, 4))

def create_green_yellow_colormap(n_classes=8):
    
    colors = [
        '#008000',  
        '#40E080',  
        '#E0E060',  
        '#B8860B'   
    ]
    
    if n_classes != 8:
        cmap = LinearSegmentedColormap.from_list('green_yellow', colors, N=n_classes)
        return [cmap(i/(n_classes-1)) for i in range(n_classes)]
    else:
        return [hex_to_rgb(color) for color in colors]

def plot_categorical_continuous_map_with_legend(
    continuous_path,  
    categorical_path, 
    custom_colors0=None,  
    custom_colors1=None,  
    vmin0=0, vmax0=320,     
    vmin1=0, vmax1=320,     
    n_classes=8,        
    world_map_path="world.shp",   
    output_map_path="output_map.png",
    label="",
    label0="Conventional",  
    label1="Enhanced",
    legend_position=(0.05, 0.08),  # 左下角位置(x, y)，默认在整个图的5%位置
    legend_size=0.25,  # 图例占整个图的比例，默认25%
    figsize=(15, 10),
    dpi=1000,
    labelal=None
):

    # 1. 读取数据
    world_map = gpd.read_file(world_map_path)
    
    with rasterio.open(continuous_path) as src_cont, \
         rasterio.open(categorical_path) as src_cat:
        
        if src_cont.crs != world_map.crs:
            world_map = world_map.to_crs(src_cont.crs)
        
        cont_data = src_cont.read(1, masked=True)
        cat_data = src_cat.read(1, masked=True)
        transform = src_cont.transform
        
        mask = cont_data.mask | cat_data.mask
        cont_data = np.where(mask, np.nan, cont_data.data)
        cat_data = np.where(mask, np.nan, cat_data.data)
    
    # 2. 创建颜色映射
    # 使用自定义色带或默认色带
    if custom_colors0 is not None:
        colors0 = custom_colors0
        if len(colors0) != n_classes:
            # 如果自定义色带长度不匹配，进行插值
            cmap = LinearSegmentedColormap.from_list('custom0', colors0, N=n_classes)
            colors0 = [cmap(i/(n_classes-1)) for i in range(n_classes)]
    else:
        # 默认色带：蓝到深蓝
        colors0 = [(
            0.6 + (0.1-0.6)*i/(n_classes-1),
            0.1 + (0.3-0.1)*i/(n_classes-1),
            0.8 + (0.8-0.8)*i/(n_classes-1)
        ) for i in range(n_classes)]
    
    if custom_colors1 is not None:
        colors1 = custom_colors1
        if len(colors1) != n_classes:
            # 如果自定义色带长度不匹配，进行插值
            cmap = LinearSegmentedColormap.from_list('custom1', colors1, N=n_classes)
            colors1 = [cmap(i/(n_classes-1)) for i in range(n_classes)]
    else:
        # 默认色带：绿到红
        colors1 = [(
            0.001 + (0.5-0.001)*i/(n_classes-1),
            0.4 + (0.5-0.4)*i/(n_classes-1),
            0.001 + (0.002-0.001)*i/(n_classes-1)
        ) for i in range(n_classes)]
    
    cmap = ListedColormap(colors0 + colors1 + [(0,0,0,0)])  # 添加透明色
    
    # 3. 数据标准化
    norm_data0 = np.floor(((cont_data - vmin0) / (vmax0 - vmin0)) * (n_classes-1))
    norm_data1 = np.floor(((cont_data - vmin1) / (vmax1 - vmin1)) * (n_classes-1))
    
    norm_data0 = np.clip(norm_data0, 0, n_classes-1).astype(int)
    norm_data1 = np.clip(norm_data1, 0, n_classes-1).astype(int)
    
    combined_index = np.where(cat_data == 0, norm_data0, norm_data1 + n_classes)
    combined_index = np.where(mask, 2*n_classes, combined_index)
    
    # 4. 绘制地图
    fig, ax = plt.subplots(figsize=figsize)
    world_map.boundary.plot(ax=ax, edgecolor='gray', linewidth=0.8)
    # 保存完整图片
    plt.text(0.02, 0.97, labelal, transform=plt.gca().transAxes, 
         fontsize=30, fontweight='bold', 
         verticalalignment='top', horizontalalignment='left')
    
    show(
        combined_index,
        transform=transform,
        ax=ax,
        cmap=cmap,
        vmin=0,
        vmax=2*n_classes,
        interpolation='nearest'
    )
    
    ax.set_ylim(-58, 90)
    ax.set_xlim(-180, 180)
    ax.set_axis_off()
    
    # 5. 在左下角添加图例
    # 获取轴的位置和大小
    pos = ax.get_position()
    
    # 计算图例在图中的位置和大小
    legend_x = pos.x0 + pos.width * legend_position[0]
    legend_y = pos.y0 + pos.height * legend_position[1]
    legend_width = pos.width * legend_size
    legend_height = pos.height * legend_size * 1.1   # 高度设为宽度的50%，适合上下排列的图例
    
    # 添加一个新的轴用于图例
    legend_ax = fig.add_axes([legend_x, legend_y, legend_width, legend_height])
    
    # 在新轴上绘制图例
    create_stacked_legend_inset(
        legend_ax, 
        colors0, colors1,
        int(vmin0), int(vmax0), int(vmin1), int(vmax1),
        label, label0, label1
    )
    
    # 设置白色背景，使图例更清晰
    legend_ax.patch.set_alpha(0.7)
    legend_ax.patch.set_facecolor('white')
    
    plt.savefig(output_map_path, dpi=dpi, bbox_inches='tight', pad_inches=0)
    plt.close()

def create_stacked_legend_inset(ax, colors0, colors1,
                              vmin0, vmax0, vmin1, vmax1,
                              label, label0, label1):
    """
    创建上下排列的双色带图例作为地图的内嵌图例
    """
    # 参数设置
    bar_height = 0.25  # 色带高度
    bar_width = 1.0    # 色带宽度
    spacing = 1.2      # 两个色带之间的间距
    start_x = 0.01     # 起始x位置
    
    # 类型0色带（上部）
    y0 = 1.1  # 上部色带Y位置
    for i in range(len(colors0)):
        rect = Rectangle(
            xy=(start_x + i*(bar_width/len(colors0)), y0),
            width=bar_width/len(colors0),
            height=bar_height,
            facecolor=colors0[i],
            edgecolor='none'
        )
        ax.add_patch(rect)
    
    # 类型0标签（色带正上方）
    # ax.text(start_x + bar_width/2, y0 + bar_height + 0.05, label0,
    #         ha='left', va='bottom', fontsize=20)
    ax.text(start_x + bar_width/2, y0 + bar_height + 0.77, label,
            ha='center', va='center', fontsize=30)
    
    # 类型0刻度（紧贴色带下方）
    for i, val in enumerate(np.linspace(vmin0, vmax0, 5)):
        x_pos = start_x + (i/4) * bar_width
        ax.text(x_pos, y0 - 0.1, f"{int(val)}",
                ha='center', va='top', fontsize=18)
    
    # 类型1色带（下部） y0 - bar_height - spacing - 0.5
    y1 = 0 # 下部色带Y位置
    for i in range(len(colors1)):
        rect = Rectangle(
            xy=(start_x + i*(bar_width/len(colors1)), y1),
            width=bar_width/len(colors1),
            height=bar_height,
            facecolor=colors1[i],
            edgecolor='none',
            zorder=0
        )
        ax.add_patch(rect)
    
    ax.text(start_x + bar_width/2, y0 + bar_height + 0.25, label0,
            ha='center', va='center', fontsize=18)
    # 类型1标签（色带正上方）
    ax.text(start_x + bar_width/2, y1 + bar_height + 0.25, label1,
            ha='center', va='center', fontsize=18)
    
    # 类型1刻度（紧贴色带下方）
    for i, val in enumerate(np.linspace(vmin1, vmax1, 5)):
        x_pos = start_x + (i/4) * bar_width
        ax.text(x_pos, y1 - 0.1, f"{int(val)}",
                ha='center', va='top', fontsize=18)
    
    # 添加外框（可选）
    for y_pos in [y0, y1]:
        rect = Rectangle(
            xy=(start_x, y_pos),
            width=bar_width,
            height=bar_height,
            fill=False,
            edgecolor='black',
            linewidth=0.8
        )
        ax.add_patch(rect)
    
    # 或者使用更保守的范围
     # 计算合适的y轴范围，确保所有元素都可见
    # min_y = min(y1 - 0.2, -0.2)  # 考虑刻度文本的下方空间
    # max_y = max(y0 + bar_height + 0.2, 1.2)  # 考虑标签文本的上方空间
    
    ax.set_xlim(0, 1.1)  # 稍微扩大x轴范围确保右侧不被裁剪
    ax.set_ylim(-0.4, 1.5)  # 动态调整y轴范围
    ax.set_axis_off()

# 示例使用
continuous_variable = "IN"
type_variable = "TYPE_IN"

# 定义自定义色带（十六进制）
Nloss_colors = [
    '#C33A3B',     # 深红色
    '#DC5046',     # 中等红色
    '#F27864',     # 浅红色
    '#FFA08C',     # 中等粉红
    '#FFC8B4',     # 浅粉红
    '#A8DDE8',     # 蓝绿色
    '#96C8DC',     # 稍深的蓝绿色
    '#82B4D2',     # 蓝绿色→蓝色过渡
    '#6EA0C8',     # 中等蓝色
    '#558CBE',     # 深蓝过渡
]

# 转换为RGB格式（0-1范围）
Nloss_colors_rgb = [hex_to_rgb(color) for color in Nloss_colors]
Nloss_colors_rgb.reverse()  # 反转色带顺序

# 创建绿色到黄色的色带
green_yellow_colors = create_green_yellow_colormap(n_classes=10)



plot_categorical_continuous_map_with_legend(
    continuous_path=f"tiffs/final_map/change_IN.tif",
    categorical_path=f"tiffs/final_map/final_TYPE_IN.tif",
    custom_colors0=Nloss_colors_rgb,  # 使用自定义色带
    custom_colors1=green_yellow_colors,  # 使用绿色到黄色色带
    vmin0=-80, vmax0=80,
    vmin1=-80, vmax1=80,
    n_classes=8,  # 注意：Nloss_colors有10个颜色，所以这里设为10
    world_map_path="../../../data/geo_dataset/worldMap/世界国家分布.shp",
    output_map_path=f"save/final/final_delta_{continuous_variable}_{type_variable}.png",
    label="Δ IN (kg N ha$^{{-1}}$)",
    label0="Remain conventional",
    label1="Replaced with SRF",
    legend_position=(0.02, 0.01),  # 图例位置
    legend_size=0.25,  # 图例大小
    labelal='a'
)

C:\Users\JZ\AppData\Local\Temp\ipykernel_14304\642790682.py:107: RuntimeWarning: invalid value encountered in cast
  norm_data0 = np.clip(norm_data0, 0, n_classes-1).astype(int)
C:\Users\JZ\AppData\Local\Temp\ipykernel_14304\642790682.py:108: RuntimeWarning: invalid value encountered in cast
  norm_data1 = np.clip(norm_data1, 0, n_classes-1).astype(int)


In [9]:
def create_legend_inset(ax, colors, vmin, vmax, label, isLog=False, n_classes=8):
    """
    创建内嵌图例
    """
    # 参数设置
    bar_height = 0.8  # 色带高度
    bar_width = 1.0    # 色带宽度
    start_x = 0.01     # 起始x位置
    y_pos = 0.6        # 色带Y位置
    
    # 绘制色带
    for i in range(len(colors)):
        rect = Rectangle(
            xy=(start_x + i*(bar_width/len(colors)), y_pos),
            width=bar_width/len(colors),
            height=bar_height,
            facecolor=colors[i],
            edgecolor='none'
        )
        ax.add_patch(rect)
    
    # 添加标签
    ax.text(start_x, y_pos + bar_height + 0.12, label,
            ha='left', va='bottom', fontsize=30)
    
    # 添加刻度 - 根据isLog参数选择不同的刻度标注方式
    if isLog:
        # 对数刻度：0, 10^0, 10^1, ..., 10^(n_classes)
        for i in range(n_classes + 1):
            x_pos = start_x + (i/n_classes) * bar_width
            if i == 0:
                tick_label = "0"
            else:
                tick_label = f"10$^{{{i}}}$"  # 使用LaTeX格式创建上标
            ax.text(x_pos, y_pos - 0.2, tick_label,
                    ha='center', va='top', fontsize=18)
    else:
        # 线性刻度
        for i, val in enumerate(np.linspace(vmin, vmax, 5)):
            x_pos = start_x + (i/4) * bar_width
            ax.text(x_pos, y_pos - 0.2, f"{int(val)}",
                    ha='center', va='top', fontsize=18)
    
    # 添加外框
    rect = Rectangle(
        xy=(start_x, y_pos),
        width=bar_width,
        height=bar_height,
        fill=False,
        edgecolor='black',
        linewidth=0.8
    )
    ax.add_patch(rect)
    
    ax.set_xlim(0, 1.1)
    ax.set_ylim(-0.1, 1.6)
    ax.set_axis_off()


def plot_single_continuous_map_with_legend(
    input_path,          # 输入TIFF路径
    custom_colors,        # 色带范围 [(起始色), (结束色)]
    vmin, vmax,          # 值范围
    n_classes=8,         # 颜色分段数
    world_map_path="world.shp",   # 世界地图路径
    output_map_path="output_map.png",  # 地图输出路径
    label="Variable",    # 图例标签
    legend_position=(0.05, 0.15),  # 图例位置(x, y)，默认在整个图的左下角5%位置
    legend_size=0.25,    # 图例占整个图的比例，默认25%
    figsize=(15, 10),    # 地图尺寸
    dpi=1000,            # 输出分辨率
    isLog=False,
    labelal=None
):
    """
    绘制单连续变量地图，图例集成在左下角
    """
    # 1. 读取世界地图
    world_map = gpd.read_file(world_map_path)
    
    # 2. 读取栅格数据
    with rasterio.open(input_path) as src:
        # 确保CRS匹配
        if src.crs != world_map.crs:
            world_map = world_map.to_crs(src.crs)
        
        # 读取数据
        data = src.read(1, masked=True)
        transform = src.transform
        
        # 处理NaN值
        data = np.where(data.mask, np.nan, data.data)
        if isLog:
            data = np.where(data < 0, np.nan, data)
            data = np.where(data > 0, np.log10(data), np.nan)
    
    # 3. 创建颜色映射
    if custom_colors is not None:
        colors = custom_colors
        if len(colors) != n_classes:
            # 如果自定义色带长度不匹配，进行插值
            cmap = LinearSegmentedColormap.from_list('custom0', colors, N=n_classes)
            colors = [cmap(i/(n_classes-1)) for i in range(n_classes)]
    else:
        raise ValueError
    
    cmap = LinearSegmentedColormap.from_list("custom", colors, N=n_classes)
    
    # 4. 绘制地图
    fig, ax = plt.subplots(figsize=figsize)
    world_map.boundary.plot(ax=ax, edgecolor='gray', linewidth=0.8)
    plt.text(0.02, 0.97, labelal, transform=plt.gca().transAxes, 
         fontsize=30, fontweight='bold', 
         verticalalignment='top', horizontalalignment='left')
    # 使用rasterio的show函数
    show(
        data,
        transform=transform,
        ax=ax,
        cmap=cmap,
        vmin=vmin,
        vmax=vmax,
        interpolation='nearest'
    )
    
    ax.set_ylim(-58, 90)
    ax.set_xlim(-180, 180)
    ax.set_axis_off()
    
    # 5. 在左下角添加图例
    # 获取轴的位置和大小
    pos = ax.get_position()
    
    # 计算图例在图中的位置和大小
    legend_x = pos.x0 + pos.width * legend_position[0]
    legend_y = pos.y0 + pos.height * legend_position[1]
    legend_width = pos.width * legend_size
    legend_height = pos.height * legend_size * 0.3  # 高度设为宽度的30%，适合单色带图例
    
    # 添加一个新的轴用于图例
    legend_ax = fig.add_axes([legend_x, legend_y, legend_width, legend_height])
    
    # 在新轴上绘制图例
    if isLog:
        create_legend_inset(
            legend_ax, 
            colors,
            0, n_classes,
            label, isLog=isLog, n_classes=n_classes
        )
    else:
        create_legend_inset(
            legend_ax, 
            colors,
            int(vmin), int(vmax),
            label, isLog=isLog, n_classes=n_classes
        )
    
    # 设置白色背景，使图例更清晰
    legend_ax.patch.set_alpha(0.7)
    legend_ax.patch.set_facecolor('white')
    
    # 保存完整图片
    plt.savefig(output_map_path, dpi=dpi, bbox_inches='tight', pad_inches=0)
    plt.close()

In [10]:
continuous_variable_dict = {"change_ON": {"color": Nloss_colors, "range": [-10, 10], "label": "Δ ON (kg N ha$^{{-1}}$)", "labelal":'b'}, 
                            "change_IP": {"color": Nloss_colors, "range": [-40, 40], "label": "Δ IP (kg P ha$^{{-1}}$)", "labelal":'c'},
                            "change_IK": {"color": Nloss_colors, "range": [-60, 60], "label": "Δ IK (kg K ha$^{{-1}}$)", "labelal":'d'}}


for variable, info_dict in continuous_variable_dict.items():
    
    color_list = info_dict["color"]
    vmin, vmax = info_dict["range"]
    label = info_dict["label"]

    plot_single_continuous_map_with_legend(
        input_path=f"tiffs/final_map/{variable}.tif",
        custom_colors=color_list,  # 浅蓝到深蓝
        vmin=vmin, 
        vmax=vmax,
        label=label,
        output_map_path=f"save/final/{variable}.png",
        world_map_path="../../../data/geo_dataset/worldMap/世界国家分布.shp",
        legend_position=(0.02, 0.10),  # 左下角位置(x, y)
        legend_size=0.25,  # 图例占整个图的比例
        dpi=300,
        labelal=info_dict["labelal"]
    )
   

In [11]:
continuous_variable_dict = {"final_BC": {"color": [(0.85, 0.92, 1.00), (0.1, 0.3, 0.8)], "range": [0, 4], "label": "BC (kg C ha$^{{-1}}$)", "labelal":'e'}} 
for variable, info_dict in continuous_variable_dict.items():
    
    color_list = info_dict["color"]
    vmin, vmax = info_dict["range"]
    label = info_dict["label"]

    plot_single_continuous_map_with_legend(
        input_path=f"tiffs/final_map/{variable}.tif",
        custom_colors=color_list,  # 浅蓝到深蓝
        vmin=vmin, 
        vmax=vmax,
        label=label,
        n_classes=4,
        output_map_path=f"save/final/{variable}.png",
        world_map_path="../../../data/geo_dataset/worldMap/世界国家分布.shp",
        legend_position=(0.02, 0.10),  # 左下角位置(x, y)
        legend_size=0.25,  # 图例占整个图的比例
        isLog=True,
        dpi=1000,
        labelal=info_dict["labelal"]
    )

C:\Users\JZ\AppData\Local\Temp\ipykernel_14304\2748337253.py:95: RuntimeWarning: divide by zero encountered in log10
  data = np.where(data > 0, np.log10(data), np.nan)


In [12]:
"""
This portion of code has been deprecated. However, if readers wish to visualize the interaction between two Boolean variables in a 2x2 configuration world map, they may use this code. Please note that the author does not guarantee its correctness.
"""
def plot_boolean_bivariate_map_with_legend(
    raster_path1, raster_path2, 
    color_bounds1, color_bounds2,
    fig_label, 
    bool_labels1,
    bool_labels2,
    world_map_path,
    output_map_path,
    legend_position=(0.05, 0.05),  # 左下角位置(x, y)，默认在整个图的5%位置
    legend_size=0.25,  # 图例占整个图的比例，默认25%
    labelal=None
):
    """
    绘制布尔型双变量地图(0-1变量)，图例集成在左下角
    """
    # 1. 读取世界地图
    world_map = gpd.read_file(world_map_path)
    
    # 2. 读取并处理栅格数据
    with rasterio.open(raster_path1) as src1, rasterio.open(raster_path2) as src2:
        # 确保CRS匹配
        if src1.crs != world_map.crs:
            world_map = world_map.to_crs(src1.crs)
        
        # 用世界地图边界裁剪数据，使用float32类型读取
        data1 = src1.read(1, masked=True)
        data2 = src2.read(1, masked=True)
        
        # 转换为布尔型(0或1)
        bool_data1 = np.where(data1 > 0.5, 1, 0)  # >0.5视为True(1)
        bool_data2 = np.where(data2 > 0.5, 1, 0)  # >0.5视为True(1)
        
        # 处理NaN值
        bool_data1 = np.where(data1.mask | (data1 < 0), -1, bool_data1)
        bool_data2 = np.where(data2.mask | (data2 < 0), -1, bool_data2)
        
        # 创建2x2颜色矩阵(布尔型只有4种组合)
        colors = [(color_bounds1[0][0] * 0.5 + color_bounds2[0][0] * 0.5,
                   color_bounds1[0][1] * 0.5 + color_bounds2[0][1] * 0.5,
                   color_bounds1[0][2] * 0.5 + color_bounds2[0][2] * 0.5),
                  (color_bounds1[0][0] * 0.5 + color_bounds2[1][0] * 0.5,
                   color_bounds1[0][1] * 0.5 + color_bounds2[1][1] * 0.5,
                   color_bounds1[0][2] * 0.5 + color_bounds2[1][2] * 0.5),
                  (color_bounds1[1][0] * 0.5 + color_bounds2[0][0] * 0.5,
                   color_bounds1[1][1] * 0.5 + color_bounds2[0][1] * 0.5,
                   color_bounds1[1][2] * 0.5 + color_bounds2[0][2] * 0.5),
                  (color_bounds1[1][0] * 0.5 + color_bounds2[1][0] * 0.5,
                   color_bounds1[1][1] * 0.5 + color_bounds2[1][1] * 0.5,
                   color_bounds1[1][2] * 0.5 + color_bounds2[1][2] * 0.5), 
                  (0, 0, 0, 0)]  # 添加透明色用于NaN值
        
        bivariate_cmap = ListedColormap(colors)
        
        # 创建组合索引 (0-3对应4种布尔组合)
        combined_index = bool_data1 * 2 + bool_data2
        combined_index = np.where((bool_data1 == -1) | (bool_data2 == -1), 4, combined_index)  # NaN值设为4
        
        # 3. 绘制主地图
        fig, ax = plt.subplots(figsize=(15, 10))
        
        # 绘制国家边界
        world_map.boundary.plot(ax=ax, edgecolor='gray', linewidth=0.8)
        plt.text(0.02, 0.97, labelal, transform=plt.gca().transAxes, 
         fontsize=30, fontweight='bold', 
         verticalalignment='top', horizontalalignment='left')
        
        # 获取地理变换信息
        transform = src1.transform
        
        # 绘制双变量数据
        show(
            combined_index,
            transform=transform,
            ax=ax,
            cmap=bivariate_cmap,
            vmin=0,
            vmax=4,
            interpolation='nearest'
        )
                
        # 设置显示范围
        ax.set_ylim(-58, 90)
        ax.set_xlim(-180, 180)
        ax.set_axis_off()
        
        # 4. 在左下角添加图例
        # 获取轴的位置和大小
        pos = ax.get_position()
        
        # 计算图例在图中的位置和大小
        legend_x = pos.x0 + pos.width * legend_position[0]
        legend_y = pos.y0 + pos.height * legend_position[1]
        legend_width = pos.width * legend_size
        legend_height = pos.height * legend_size
        
        # 添加一个新的轴用于图例
        legend_ax = fig.add_axes([legend_x, legend_y, legend_width, legend_height])
        
        # 在新轴上绘制图例
        create_boolean_legend_inset(
            legend_ax,
            color_bounds1, color_bounds2,
            fig_label,
            bool_labels1, bool_labels2
        )
        
        # 设置白色背景，使图例更清晰
        legend_ax.patch.set_alpha(0.7)
        legend_ax.patch.set_facecolor('white')
        
        # 保存完整图片
        plt.savefig(output_map_path, dpi=1000, bbox_inches='tight', pad_inches=0)
        plt.close()

def create_boolean_legend_inset(ax, color_bounds1, color_bounds2, fig_label, bool_labels1, bool_labels2):
    """创建布尔型双变量图例作为地图的内嵌图例"""
    
    # 定义四个象限的颜色
    colors = [
        # 左下: 变量1=0, 变量2=0
        (color_bounds1[0][0]*0.5 + color_bounds2[0][0]*0.5,
         color_bounds1[0][1]*0.5 + color_bounds2[0][1]*0.5,
         color_bounds1[0][2]*0.5 + color_bounds2[0][2]*0.5),
        # 右下: 变量1=0, 变量2=1
        (color_bounds1[0][0]*0.5 + color_bounds2[1][0]*0.5,
         color_bounds1[0][1]*0.5 + color_bounds2[1][1]*0.5,
         color_bounds1[0][2]*0.5 + color_bounds2[1][2]*0.5),
        # 左上: 变量1=1, 变量2=0
        (color_bounds1[1][0]*0.5 + color_bounds2[0][0]*0.5,
         color_bounds1[1][1]*0.5 + color_bounds2[0][1]*0.5,
         color_bounds1[1][2]*0.5 + color_bounds2[0][2]*0.5),
        # 右上: 变量1=1, 变量2=1
        (color_bounds1[1][0]*0.5 + color_bounds2[1][0]*0.5,
         color_bounds1[1][1]*0.5 + color_bounds2[1][1]*0.5,
         color_bounds1[1][2]*0.5 + color_bounds2[1][2]*0.5),
    ]
    
    # 绘制2x2网格的正方形
    square_size = 0.8
    start_x, start_y = 0.1, 0.1
    cell_size = square_size / 2
    
    # 绘制四个小正方形
    for i in range(2):
        for j in range(2):
            # 计算位置索引 (0=左下, 1=右下, 2=左上, 3=右上)
            idx = i * 2 + j
            rect = Rectangle(
                (start_x + j*cell_size, start_y + i*cell_size),
                cell_size, cell_size,
                facecolor=colors[idx],
                edgecolor='black',
                linewidth=1
            )
            ax.add_patch(rect)
    
    # 添加标签 - 调整字体大小适应内嵌图例
    font_size = 16
    
    # 变量1标签（放在左侧两个颜色块的中间）
    ax.text(start_x + cell_size, start_y + cell_size + 0.45, fig_label,
            ha='center', va='bottom', fontsize=18)
    
    ax.text(start_x - 0.05, start_y + cell_size*0.5, 
            bool_labels1[0], ha='right', va='center', 
            fontname='Arial', fontsize=font_size)
    ax.text(start_x - 0.05, start_y + cell_size*1.5, 
            bool_labels1[1], ha='right', va='center', 
            fontname='Arial', fontsize=font_size)
    
    # 变量2标签（放在下方两个颜色块的中间）
    ax.text(start_x + cell_size*0.5, start_y - 0.05, 
            bool_labels2[0], ha='center', va='top', 
            fontname='Arial', fontsize=font_size)
    ax.text(start_x + cell_size*1.5, start_y - 0.05, 
            bool_labels2[1], ha='center', va='top', 
            fontname='Arial', fontsize=font_size)
    
    # 设置图形范围
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_axis_off()
    ax.set_aspect('equal')

# 主程序执行部分
if __name__ == "__main__":

    variable1 = "STRAW"
    variable2 = "tillage"

    plot_boolean_bivariate_map_with_legend(
        raster_path1=f"tiffs/final_map/final_{variable1}.tif",
        raster_path2=f"tiffs/final_map/final_{variable2}.tif",
        color_bounds1=((152/255, 251/255, 152/255), (0/255, 150/255, 0/255)),  
        color_bounds2=((152/255, 251/255, 152/255), (0, 191/255, 1)),  
        fig_label="Straw and tillage management",
        bool_labels1=("NS", "SR"),  # 变量1的标签
        bool_labels2=("NT", "CT"),  # 变量2的标签
        world_map_path="../../../data/geo_dataset/worldMap/世界国家分布.shp",
        output_map_path=f"save/final/{variable1}_{variable2}.png",
        legend_position=(0.02, 0.07),  # 图例位置
        legend_size=0.25  # 图例大小
    )

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from matplotlib.colors import ListedColormap
from matplotlib.patches import Rectangle
import geopandas as gpd

def plot_onehot_maps_with_legend(
    raster_paths, colors, labels, fig_label,
    world_map_path, output_map_path,
    legend_position=(0.05, 0.05),  # 左下角位置(x, y)，默认在整个图的5%位置
    legend_size=0.25,  # 图例占整个图的比例，默认25%
    figsize=(15, 10), dpi=300, labelal=None
):
    """
    绘制四个独热编码变量的地图，图例集成在左下角
    
    参数:
        raster_paths: 四个tif文件路径的列表
        colors: 四种颜色的列表，格式[(R,G,B), ...] (0-1范围)
        labels: 四个标签文本的列表
        world_map_path: 世界地图shp文件路径
        output_map_path: 地图输出路径
        legend_position: 图例在图中的位置(x, y)，值范围0-1
        legend_size: 图例占整个图的比例，值范围0-1
        figsize: 图像大小
        dpi: 输出分辨率
    """
    # 1. 读取世界地图
    world_map = gpd.read_file(world_map_path)
    
    # 2. 读取并处理栅格数据
    data_list = []
    transform = None
    with rasterio.open(raster_paths[0]) as src:
        transform = src.transform
        if src.crs != world_map.crs:
            world_map = world_map.to_crs(src.crs)
    
    for path in raster_paths:
        with rasterio.open(path) as src:
            data = src.read(1, masked=True)
            data = np.where(data > 0.5, 1, 0)  # 转换为布尔值
            data_list.append(data)
    
    # 3. 创建颜色映射
    cmap_colors = [(0, 0, 0, 0)]  # 第一个颜色为透明(背景)
    cmap_colors.extend(colors)  # 添加四种颜色
    cmap = ListedColormap(cmap_colors)
    
    # 4. 创建组合索引 (0=背景, 1-4=四个类别)
    combined = np.zeros_like(data_list[0], dtype=int)
    for i, data in enumerate(data_list, start=1):
        combined[data == 1] = i
    
    # 5. 绘制主地图
    fig, ax = plt.subplots(figsize=figsize)
    world_map.boundary.plot(ax=ax, edgecolor='gray', linewidth=0.8)
    plt.text(0.02, 0.97, labelal, transform=plt.gca().transAxes, 
         fontsize=30, fontweight='bold', 
         verticalalignment='top', horizontalalignment='left')
    
    show(
        combined,
        transform=transform,
        ax=ax,
        cmap=cmap,
        vmin=0,
        vmax=4,
        interpolation='nearest'
    )
    
    ax.set_ylim(-58, 90)
    ax.set_xlim(-180, 180)
    ax.set_axis_off()
    
    # 6. 在左下角添加图例
    # 获取轴的位置和大小
    pos = ax.get_position()
    
    # 计算图例在图中的位置和大小
    legend_x = pos.x0 + pos.width * legend_position[0]
    legend_y = pos.y0 + pos.height * legend_position[1]
    legend_width = pos.width * legend_size
    legend_height = pos.height * legend_size * 0.3  # 高度设为宽度的30%，适合横向图例
    
    # 添加一个新的轴用于图例
    legend_ax = fig.add_axes([legend_x, legend_y, legend_width, legend_height])
    
    # 在新轴上绘制图例
    create_onehot_legend_inset(legend_ax, colors, labels, fig_label)
    
    # 设置白色背景，使图例更清晰
    legend_ax.patch.set_alpha(0.7)
    legend_ax.patch.set_facecolor('white')
    
    # 保存完整图片
    plt.savefig(output_map_path, dpi=dpi, bbox_inches='tight', pad_inches=0)
    plt.close()

def create_onehot_legend_inset(ax, colors, labels, fig_label):
    """
    创建独热编码图例作为地图的内嵌图例
    
    参数:
        ax: 图例所在的轴对象
        colors: 颜色列表 [(R,G,B), ...]
        labels: 标签列表
    """
    # 参数设置
    total_width = 0.9  # 总宽度（占图例轴比例）
    bar_height = 0.4   # 色带高度
    start_x = 0.05     # 起始x位置
    start_y = 0.3      # 起始y位置
    
    # 计算每个色带的宽度
    num_colors = len(colors)
    bar_width = total_width / num_colors
    
    # 绘制连续色带（共享边界）
    for i, (color, label) in enumerate(zip(colors, labels)):
        x_pos = start_x + i * bar_width
        
        # 绘制色带（无间隔）
        rect = Rectangle(
            (x_pos, start_y), bar_width, bar_height,
            facecolor=color,
            edgecolor='black',
            linewidth=1,
            joinstyle='miter'  # 确保直角连接
        )
        ax.add_patch(rect)
        
        # 在色带下方中央添加标签
        ax.text(
            x_pos + bar_width/2, start_y - 0.22,
            label,
            ha='center', va='top',
            fontname='Arial',
            fontsize=18,  # 调整为适合内嵌图例的大小
            color='black'
        )
    ax.text(
            start_x, start_y + 1.6,
            fig_label,
            ha='left', va='top',
            fontname='Arial',
            fontsize=30,  # 调整为适合内嵌图例的大小
            color='black',
        )
    # 添加整体外框（可选）
    outline = Rectangle(
        (start_x, start_y), total_width, bar_height,
        fill=False, edgecolor='black', linewidth=1.5
    )
    ax.add_patch(outline)
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_axis_off()

# 主程序执行部分
if __name__ == "__main__":
    plot_onehot_maps_with_legend(
        raster_paths=[
            f"tiffs/final_map/final_WM_CF.tif",
            f"tiffs/final_map/final_WM_MD.tif",
            f"tiffs/final_map/final_WM_AWD.tif", 
            f"tiffs/final_map/final_WM_RF.tif",
        ],
        colors=[
            (0.35, 0.70, 0.90),  # 天空蓝 - 常规灌溉
            (0.80, 0.40, 0.60),   # 紫红色 - 其他类型
            (0.90, 0.60, 0.00),  # 琥珀色 - AWD（面积最大，最醒目）
            (0.20, 0.60, 0.30),  # 森林绿 - 雨养农田    
        ],
        labels=["CF", "MD", "II", "RF"],
        fig_label="Irrigation mode",
        world_map_path="../../../data/geo_dataset/worldMap/世界国家分布.shp",
        output_map_path=f"save/final/water.png",
        legend_position=(0.02, 0.1),  # 图例位置
        legend_size=0.25,  # 图例大小
        labelal='f'
    )

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from matplotlib.colors import ListedColormap
from matplotlib.patches import Rectangle
import geopandas as gpd

def plot_binary_maps_with_legend(
    raster_path, colors, labels, fig_label,
    world_map_path="../../../data/geo_dataset/worldMap/世界国家分布.shp",
    output_map_path="save/final/tillage.png",
    legend_position=(0.02, 0.1),  # 左下角位置(x, y)，默认在整个图的5%位置
    legend_size=0.25,  # 图例占整个图的比例，默认25%
    figsize=(15, 10), dpi=300,
    value_labels=None,  # 自定义1和0在图例中显示的内容
    labelal=None
):
    """
    绘制二值数据的地图，图例集成在左下角
    
    参数:
        raster_path: tif文件路径
        colors: 两种颜色的列表，格式[(R,G,B), ...] (0-1范围)，分别对应0和1
        labels: 两个标签文本的列表，分别对应0和1
        fig_label: 图例标题
        world_map_path: 世界地图shp文件路径
        output_map_path: 地图输出路径
        legend_position: 图例在图中的位置(x, y)，值范围0-1
        legend_size: 图例占整个图的比例，值范围0-1
        figsize: 图像大小
        dpi: 输出分辨率
        value_labels: 可选，自定义1和0在图例中显示的内容，格式为字典{0: "标签0", 1: "标签1"}
    """
    # 设置默认的value_labels
    if value_labels is None:
        value_labels = {0: "0", 1: "1"}
    
    # 1. 读取世界地图
    world_map = gpd.read_file(world_map_path)
    
    # 2. 读取并处理栅格数据
    with rasterio.open(raster_path) as src:
        transform = src.transform
        data = src.read(1, masked=True)
        
        # 转换为二值数据，保持空值不变
        data_binary = np.where(data > 0.5, 1, 0)
        data_binary = np.ma.masked_where(data.mask, data_binary)  # 保持原有的掩码
    
    # 3. 创建颜色映射 (0=透明背景, 1=第一种颜色, 2=第二种颜色)
    cmap_colors = [(0, 0, 0, 0)]  # 第一个颜色为透明(背景)
    cmap_colors.extend(colors)  # 添加两种颜色
    cmap = ListedColormap(cmap_colors)
    
    # 4. 调整数据值 (0=背景, 1=第一个类别, 2=第二个类别)
    combined = np.zeros_like(data_binary, dtype=int)
    combined[data_binary == 0] = 1  # 对应colors中的第一个颜色
    combined[data_binary == 1] = 2  # 对应colors中的第二个颜色
    combined = np.ma.masked_where(data_binary.mask, combined)  # 保持掩码
    
    # 5. 绘制主地图
    fig, ax = plt.subplots(figsize=figsize)
    world_map.boundary.plot(ax=ax, edgecolor='gray', linewidth=0.8)
    plt.text(0.02, 0.97, labelal, transform=plt.gca().transAxes, 
         fontsize=30, fontweight='bold', 
         verticalalignment='top', horizontalalignment='left')
    
    show(
        combined,
        transform=transform,
        ax=ax,
        cmap=cmap,
        vmin=0,
        vmax=2,
        interpolation='nearest'
    )
    
    ax.set_ylim(-58, 90)
    ax.set_xlim(-180, 180)
    ax.set_axis_off()
    
    # 6. 在左下角添加图例
    # 获取轴的位置和大小
    pos = ax.get_position()
    
    # 计算图例在图中的位置和大小
    legend_x = pos.x0 + pos.width * legend_position[0]
    legend_y = pos.y0 + pos.height * legend_position[1]
    legend_width = pos.width * legend_size
    legend_height = pos.height * legend_size * 0.3  # 高度设为宽度的30%，适合横向图例
    
    # 添加一个新的轴用于图例
    legend_ax = fig.add_axes([legend_x, legend_y, legend_width, legend_height])
    
    # 在新轴上绘制图例
    create_binary_legend_inset(legend_ax, colors, labels, fig_label, value_labels)
    
    # 设置白色背景，使图例更清晰
    legend_ax.patch.set_alpha(0.7)
    legend_ax.patch.set_facecolor('white')
    
    # 保存完整图片
    plt.savefig(output_map_path, dpi=dpi, bbox_inches='tight', pad_inches=0)
    plt.close()

def create_binary_legend_inset(ax, colors, labels, fig_label, value_labels):
    """
    创建二值数据图例作为地图的内嵌图例
    
    参数:
        ax: 图例所在的轴对象
        colors: 颜色列表 [(R,G,B), ...]，分别对应0和1
        labels: 标签列表，分别对应0和1
        fig_label: 图例标题
        value_labels: 字典，自定义1和0在图例中显示的内容
    """
    # 参数设置
    total_width = 0.7  # 总宽度（占图例轴比例）
    bar_height = 0.4   # 色带高度
    start_x = 0.05     # 起始x位置
    start_y = 0.3      # 起始y位置
    
    # 计算每个色带的宽度
    num_colors = len(colors)
    bar_width = total_width / num_colors
    
    # 绘制色带
    for i, (color, label) in enumerate(zip(colors, labels)):
        x_pos = start_x + i * bar_width
        
        # 绘制色带（无间隔）
        rect = Rectangle(
            (x_pos, start_y), bar_width, bar_height,
            facecolor=color,
            edgecolor='black',
            linewidth=1,
            joinstyle='miter'  # 确保直角连接
        )
        ax.add_patch(rect)
        
        # 在色带下方中央添加标签
        # 使用value_labels中的自定义标签
        value = i  # 第一个颜色对应0，第二个对应1
        display_label = value_labels.get(value, str(value))
        
        ax.text(
            x_pos + bar_width/2, start_y - 0.22,
            display_label,
            ha='center', va='top',
            fontname='Arial',
            fontsize=18,
            color='black'
        )
    
    # 添加图例标题
    ax.text(
        start_x, start_y + bar_height + 0.2,
        fig_label,
        ha='left', va='bottom',
        fontname='Arial',
        fontsize=30,
        color='black'
    )
    
    # 添加整体外框（可选）
    outline = Rectangle(
        (start_x, start_y), total_width, bar_height,
        fill=False, edgecolor='black', linewidth=1.5
    )
    ax.add_patch(outline)
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_axis_off()

In [15]:
# 示例调用
plot_binary_maps_with_legend(
    raster_path="tiffs/final/final_tillage.tif",
    output_map_path="save/final/tillage.png",
    colors=[(0.35, 0.70, 0.90), (0.80, 0.40, 0.60)], 
    labels=["CT", "NT"],
    fig_label="Tillage mode",
    value_labels={0: "NT", 1: "CT"},  # 自定义图例显示内容
    labelal='g'
)

In [16]:
# 示例调用
plot_binary_maps_with_legend(
    raster_path="tiffs/final/final_STRAW.tif",
    output_map_path="save/final/STRAW.png",
    colors=[(0.35, 0.70, 0.90), (0.80, 0.40, 0.60)], 
    labels=["RT", "RM"],
    fig_label="Straw mode",
    value_labels={0: "RM", 1: "RT"},  # 自定义图例显示内容
    labelal='h'
)